In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

In [3]:
#https://www.arc.gov.au/excellence-research-australia/era-2018-journal-list
df_era = pd.read_excel('/Users/eric/Downloads/era_2018_journal_list.xlsx', sheet_name="ERA 2018 Journal List")
df_for = pd.read_excel('/Users/eric/Downloads/era_2018_journal_list.xlsx', sheet_name="FoR Codes")

In [4]:
df_for

,FoR Code,FoR Description
0,1,Mathematical Sciences
1,101,Pure Mathematics
2,102,Applied Mathematics
3,103,Numerical and Computational Mathematics
4,104,Statistics
...,...,...
174,2201,Applied Ethics
175,2202,History and Philosophy of Specific Fields
176,2203,Philosophy
177,2204,Religion and Religious Studies


In [21]:
FoR = []
FoR_dict = {}
for ix, row in df_for.iterrows():
    code = str(row['FoR Code'])
    if len(code) in [1,3]:
        code=f"0{code}"
    assert(len(code) in [2,4])

    elem = {
        "code_for": code,
        "description": row["FoR Description"].strip()
    }
    FoR_dict[code] = elem
    
for ix, row in df_for.iterrows():
    code = str(row['FoR Code'])
    if len(code) in [1,3]:
        code=f"0{code}"
        
    assert(len(code) in [2,4])
    
    code_for_2digits = code[0:2]
    description_2digits = FoR_dict[code_for_2digits]["description"]
    elem = {
        "code_for": code,
        "description": row["FoR Description"].strip(),
        "code_for_2digits": code_for_2digits,
        "description_2digits": description_2digits
    }
    FoR_dict[code] = elem
    FoR.append(elem)
    
elem = {
    "code_for": "MD",
        "description": "Multidisciplinary",
        "code_for_2digits": "MD",
        "description_2digits": "Multidisciplinary"
}
FoR_dict["MD"] = elem
FoR.append(elem)

In [22]:
df_era[["FoR 1", "FoR 2", "FoR 3", "ISSN 1", "ISSN 2", "ISSN 3", "ISSN 4", "ISSN 5", "ISSN 6", "ISSN 7"]]

,FoR 1,FoR 2,FoR 3,ISSN 1,ISSN 2,ISSN 3,ISSN 4,ISSN 5,ISSN 6,ISSN 7
0,0101,NaN,NaN,1085-3375,1687-0409,NaN,NaN,NaN,NaN,NaN
1,0802,806.0,NaN,0098-3500,1557-7295,NaN,NaN,NaN,NaN,NaN
2,0101,102.0,202.0,0167-8019,1572-9036,NaN,NaN,NaN,NaN,NaN
3,0101,802.0,NaN,0065-1036,1730-6264,NaN,NaN,NaN,NaN,NaN
4,0101,NaN,NaN,0001-5962,1871-2509,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
25012,0703,NaN,NaN,2053-714X,2095-5138,NaN,NaN,NaN,NaN,NaN
25013,2002,NaN,NaN,2055-1045,NaN,NaN,NaN,NaN,NaN,NaN
25014,0601,NaN,NaN,1674-800X,1674-8018,NaN,NaN,NaN,NaN,NaN
25015,16,20.0,22.0,2330-8117,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data_for = []
for i, row in df_era[["FoR 1", "FoR 2", "FoR 3", "ISSN 1", "ISSN 2", "ISSN 3", "ISSN 4", "ISSN 5", "ISSN 6", "ISSN 7"]].iterrows():
    for code_issn in range(1,8):
        issns = []
        raw_issn = row[f"ISSN {code_issn}"]
        if pd.isnull(raw_issn):
            continue
        issn = raw_issn.strip()
        assert(len(issn) == 9)

        #print(raw_code)

In [28]:
data_for = []
for i, row in df_era[["FoR 1", "FoR 2", "FoR 3", "ISSN 1", "ISSN 2", "ISSN 3", "ISSN 4", "ISSN 5", "ISSN 6", "ISSN 7"]].iterrows():
    codes = []
    for code_ix in range(1,4):
        raw_code = row[f"FoR {code_ix}"]
        if pd.isnull(raw_code):
            continue
        if isinstance(raw_code, float):
            raw_code = str(int(raw_code))
        if len(raw_code) in [1,3]:
            raw_code="0"+raw_code
        if raw_code not in FoR_dict:
            print(f"missing {raw_code}")
        codes.append(FoR_dict[raw_code])
        
    issns = []
    for code_issn in range(1,8):
        issns = []
        raw_issn = row[f"ISSN {code_issn}"]
        if pd.isnull(raw_issn):
            continue
        issn = raw_issn.strip()
        assert(len(issn) == 9)
        issns.append(issn)
        new_line = {
                "issn": issn,
                "FoR": codes 
        }
        data_for.append(new_line)

In [30]:
json.dump(data_for, open('FoR.json', 'w'), indent=2)